# Run ext-ZINBayes

To explain how to use ext-ZINBayes we will use the Islam dataset provided in data/. The steps are all the same for any single cell dataset, however the parameterization in each step depends on what the user wants to do. 

In [1]:
from ext_zinbayes import * 
data = CSVDataset("./data/Islam", "Islam_treated.csv", labels_file="Islam_labels.csv")

### Differential expression analysis (4 steps):
### 1 - Create a ZINBayes instance.
This is the step where we define the dimension of the latent representations ($Z_i$), the number of train Monte Carlo (MC) samples and the configuration (gene-dispersion, zero-inflation, batch correction etc).
To exemplify we will create ZINBayes where the underlying model will have no gene_dispersion, no zero-inflation, no batch correction (Islam has no batches). It will use  10-dimensional cell representations, scalings ($L_i$) and 10 MC samples during training. If the user wishes to use gene-dispersion, batch correction and/or zero inflation just change the corresponding parameter to true. The same goes for the scalings in the opposite way.

### 2 - Define and fit the  ZINBayes model with the specified configuration.
In this step, we construct the specified model and find the optimal variational distributions (posterior approximations). It only requires the data, a number of iterations and the batch index for each cell (if there are batches). In this example we define 1000 iterations and no batch indexes since Islam has no batches.

In [2]:
ext_zb = ExtZINBayes(n_components=10, n_mc_samples=5, scalings=True, 
                     gene_dispersion=False, zero_inflation=False, batch_correction=False)
ext_zb.fit(data.X, max_iter=1000, batch_idx=None)

Considering cell-specific scalings.
Considering gene-specific dispersion.
1000/1000 [100%] ██████████████████████████████ Elapsed: 226s | Loss: 17785.651


### 3 - Calculate the differential scores for each gene.
In this step, average Bayes factors are calculated for each gene, using all or a subset of valid cell pairs and a given number of Monte Carlo samples for each cell and gene $\rho_{ig}$. See the preprint for more details.

In the following example we use __ALL pairs__ and 100 MC samples for each $\rho_{ig}$. 


In [22]:
scores = ext_zb.differential_expression_scores(data.cell_types, data.labels,"MEF","ESC", n_samples=100)
print(scores)

rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
[  6.55862097  -1.76177334 -14.1300128  ...   8.89106606  15.99809118
  -0.80280499]


If we wish to use a __SUBSET of pairs__, then we need to specify the number of pairs to use. 

In [9]:
scores = ext_zb.differential_expression_scores(data.cell_types, data.labels,"MEF","ESC", n_samples=100, n_pairs=500)
print(scores)

rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
[  6.28315326  -1.4402653  -13.55761519 ...   6.45037033  12.95081831
  -1.31237454]


If the dataset contains __BATCH information__, then we need to pass that information in order to take into account the batches when pairing the cells. See the preprint for more details.

To illustrate this we will <font color=blue>simulate some batch assignement</font> regarding the Islam cells and will calculate the DE scores using that information.

In [10]:
batch_names = np.array(["batch 1", "batch 2"])
batches = np.zeros((92,))
batches[24:48] = 1
batches[70:92] = 1
print(batches)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [13]:
scores = ext_zb.differential_expression_scores(data.cell_types, data.labels, "MEF", "ESC", 
                                               batch_names, batches, n_samples=100)
print(scores)

DE considering batches
rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
[  6.48432753  -1.64470043 -12.97906732 ...   7.13388748  13.12319759
  -1.80426134]


### 4 - Rank and select (optional) genes based on the DE scores.
To obtain the __RANKING with all genes__, we simply pass the names of the genes and the scores. Note that the names and scores must be in the same order. If the CSVDataset object is used then that is already fulfilled.

In [23]:
rank = ext_zb.differentially_expressed_genes(data.gene_names,scores)
print(rank.head(n=5))

        Gene     factor
6364  Dppa5a  18.277080
6390   Gsta4  18.268909
5391  Ifitm1  18.140479
2807   Cldn6  17.987808
4381    Alpl  17.975856


If we want to chose the __DEG__, a treshold must be specified (~ 2 or 3).

In [24]:
DEG = ext_zb.differentially_expressed_genes(data.gene_names,scores, threshold=3)
print(DEG.head(n=5))

        Gene     factor
6364  Dppa5a  18.277080
6390   Gsta4  18.268909
5391  Ifitm1  18.140479
2807   Cldn6  17.987808
4381    Alpl  17.975856


# Assess multiple AUC

In the paper, most of the evaluation process was based on the AUC of multiple runs. To do so we need to run the described pipeline several times. Here we exemplify the procedure to calculate the __mean AUC__ for the __Islam__ dataset using only 2 runs.

First let us define a function to calculate the AUC of each run, using the sklearn package.

In [25]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def calculate_auc(scrs, lbls, trueDEG):
    scores = np.array(scrs)
    labels = np.array(lbls.isin(trueDEG))
    fpr,tpr,thresh = roc_curve(labels, scores)
    return auc(fpr, tpr)

Now we can perfrom and calculate the AUC of the 2 runs, as well as the mean AUC 

In [26]:
n_times = 2 #number of runs
n_epochs = 1000
path = './data/Islam'
trueDEG = pd.read_csv(path + '/trueDEG.txt', sep=",", header=0)
data = CSVDataset(path, "Islam_treated.csv", labels_file="Islam_labels.csv")
aucs = []
for i in range(n_times):
    ext_zb = ExtZINBayes(n_components=10, n_mc_samples=5, scalings=True, gene_dispersion=False, 
                    zero_inflation=False, batch_correction=False) 
    ext_zb.fit(data.X, max_iter=n_epochs)
    scores = ext_zb.differential_expression_scores(data.cell_types, data.labels,"MEF","ESC", n_samples=100)
    res = ext_zb.differentially_expressed_genes(data.gene_names,scores)
    auc_zinb = calculate_auc(res['factor'], res["Gene"], trueDEG["SYMBOL"])
    aucs.append(auc_zinb)
print('AUCs: ' + str(aucs))
print('Average AUC: ' + str(np.mean(np.array(aucs))))

Considering cell-specific scalings.
Considering gene-specific dispersion.
1000/1000 [100%] ██████████████████████████████ Elapsed: 226s | Loss: 17760.743
rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
Considering cell-specific scalings.
Considering gene-specific dispersion.
1000/1000 [100%] ██████████████████████████████ Elapsed: 226s | Loss: 17761.860
rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
AUCs: [0.6325640382522362, 0.6458854615670362]
Average AUC: 0.6392247499096362


For the __synthetic__ data the procedure is almost the same, with some small changes in certain parameters.
Here we exemplify the procedure with the __50-50-50-50__ dataset.

In [10]:
n_times = 2 #number of runs
n_epochs = 1000
path = './data/Sinteticos/50-50-50-50-400-400'
trueDEG = pd.read_csv(path + '/trueDEG.txt', sep=",", header=0)
data = CSVDataset(path, "synthetic.csv", labels_file="labels.csv")
aucs = []
for i in range(n_times):
    ext_zb = ExtZINBayes(n_components=10, n_mc_samples=5, scalings=True, gene_dispersion=False, 
                        zero_inflation=False, batch_correction=False) 
    ext_zb.fit(data.X, max_iter=n_epochs)
    scores = ext_zb.differential_expression_scores(data.cell_types, data.labels,"B","A", n_samples=100)
    res = ext_zb.differentially_expressed_genes(data.gene_names,scores)
    auc_zinb = calculate_auc(res['factor'], res["Gene"], trueDEG["Gene"])
    aucs.append(auc_zinb)
print('AUCs: ' + str(aucs))
print('Average AUC: ' + str(np.mean(np.array(aucs))))

Considering cell-specific scalings.
Considering gene-specific dispersion.
1000/1000 [100%] ██████████████████████████████ Elapsed: 336s | Loss: 5348.920
rho1_values shape: (100, 500, 1000)
rho2_values shape: (100, 500, 1000)
Considering cell-specific scalings.
Considering gene-specific dispersion.
1000/1000 [100%] ██████████████████████████████ Elapsed: 336s | Loss: 5347.279
rho1_values shape: (100, 500, 1000)
rho2_values shape: (100, 500, 1000)
AUCs: [0.9508375, 0.9641500000000001]
Average AUC: 0.95749375
